In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pyrealsense2 as rs
import cv2

In [2]:
intrinsics = rs.intrinsics()
intrinsics.width = 848
intrinsics.height = 480
intrinsics.fx = 424.5785827636719
intrinsics.fy = 424.5785827636719
intrinsics.ppx = 422.18994140625
intrinsics.ppy = 244.84666442871094
intrinsics.model = rs.distortion.brown_conrady

In [3]:
data = pd.read_csv('id_weight_sex_uuid.csv')

In [4]:
ids = data.id.unique()
ids_cont = np.zeros(len(ids))

In [7]:
data.head()

,row,uuid,id,sex,weight
0,0,3b144fdfdb9f4e02b0f65aed674fbd72,0268,1,24.5
1,1,960fd92c91f541f6b85889426c39a0fe,0261,1,27.1
2,2,ca10f050724d47f5971c3530ef341a8d,0236,0,25.5
3,3,3c4dcc3c6d83425dab0d0acd75227f99,0236,0,25.5
4,4,46d3f67bbf684992ac945ce7c1991ae7,0238,1,25.8


In [59]:
for index,row in data.iterrows():
    img = np.load('depth/'+row.uuid+'.npy')
    depth_colorized = cv2.applyColorMap(np.uint8(cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)),cv2.COLORMAP_JET)
    img_color = np.load('color/'+row.uuid+'.npy')
    
    _, mask = cv2.threshold(img,900,255,cv2.THRESH_BINARY)
    mask = mask.astype(np.uint8)
    size_h, size_w = mask.shape

    cv2.imwrite('results/img_base_depth.png', depth_colorized)
    cv2.imwrite('results/img_base_color.png', img_color)
    cv2.imwrite('results/img_base_mask.png', mask)
    
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour = []
    contour_area = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>20000 and area<200000:
            if area>contour_area:
                contour_area = area
                contour = cnt
    if contour_area>500:
        x,y,w,h = cv2.boundingRect(contour)

        border = 50
        v = [x-border, x+w+border,y-border, y+h+border]
        v[0] = v[0] if v[0]>=0 else 0
        v[1] = v[1] if v[1]<size_w else size_w-1
        v[2] = v[2] if v[2]>=0 else 0
        v[3] = v[3] if v[3]<size_h else size_h-1
        
        img_bb = depth_colorized.copy()
        image = cv2.rectangle(img_bb, (x,y), (x+w,y+h), (0,255,0), 5)
        image = cv2.rectangle(img_bb, (v[0],v[2]), (v[1],v[3]), (0,255,0), 3)

        img = img[v[2]:v[3], v[0]:v[1]]
        img_color = img_color[v[2]:v[3], v[0]:v[1]]
        depth_colorized = depth_colorized[v[2]:v[3], v[0]:v[1]]
        mask = mask[v[2]:v[3], v[0]:v[1]]
        
        cv2.imwrite('results/img_base_bb.png', img_bb)
        cv2.imwrite('results/img_process_depth.png', depth_colorized)
        cv2.imwrite('results/img_process_color.png', img_color)
        cv2.imwrite('results/img_process_mask.png', mask)

    depth_value = cv2.mean(img, mask)[0]
    height = rs.rs2_deproject_pixel_to_point(intrinsics, [100,100], depth_value)[2] #xyz
    height = height / 1000
    #id_sexo_peso_altura
    index = np.where(ids==row.id)[0][0]
    sex = 'Men' if row.sex == 0 else 'Women'
    ids_cont[index] = ids_cont[index]+1
    id_cont = int(ids_cont[index])
    
    
    #cv2.imwrite('images/'+str(index+1).zfill(4)+'_'+sex+'_'+str(row.weight)+'_'+ "{:.2f}".format(height)+'_'+str(id_cont).zfill(6)+'.png', img)
    break